导入numpy包

In [9]:
from math import sqrt, pi, log, exp, floor
import numpy.matlib
import numpy as np
np.__version__

'1.19.1'

各电缆间距函数

In [25]:
def cal_dis(row, column, typeCircuit, S1, S2, S3, GMRs):
    numberCircuit = row * column  # 回路个数，n
    numberCable = 3 * numberCircuit  # 电缆数目，3×n
    coorCable = np.zeros((numberCable, 2))  # 电缆坐标，以第一根电缆为原点，向上和左为正方向

    for i in range(row):  # 行 matlab下标从1开始，numpy下标从0开始
        for j in range(column):  # 列
            temp1 = i * row + j  # 第几个回路
            temp2 = temp1 * 3  # 该回路的第一根电缆，上或者中间
            if typeCircuit[temp1][0] == 1:  # 对于三角形排列
                coorCable[temp2, 0] = j * S1
                coorCable[temp2, 1] = i * S1
                coorCable[temp2 + 1, 0] = j * S1 - 0.5 * S3
                coorCable[temp2 + 1, 1] = i * S1 - sqrt(S2**2 - (0.5 * S3)**2)
                coorCable[temp2 + 2, 0] = j * S1 + 0.5 * S3
                coorCable[temp2 + 2, 1] = i * S1 - sqrt(S2**2 - (0.5 * S3)**2)
            else:
                coorCable[temp2, 0] = j * S1
                coorCable[temp2, 1] = i * S1
                coorCable[temp2 + 1, 0] = j * S1 - S2
                coorCable[temp2 + 1, 1] = i * S1
                coorCable[temp2 + 2, 0] = j * S1 + S2
                coorCable[temp2 + 2, 1] = i * S1
    
    print('coorCable', coorCable)
    distance = np.zeros((numberCable, numberCable))

    for i in range(numberCable):
        for j in range(numberCable):
            if i != j:
                distance[i, j] = sqrt((coorCable[i, 0] - coorCable[j, 0])**2 +
                                      (coorCable[i, 1] - coorCable[j, 1])**2)
            else:
                distance[i, j] = GMRs

    return distance

初始化参数

In [26]:
row = 1  # 回路排列的行数
column = 4  # 回路排列的列数
numberCircuit = row * column  # 回路个数，n
numberCable = 3 * numberCircuit  # 电缆总数目，3×n

# 各回路的类型(列向量,三角型输入1，直线型输入2)
typeCircuit = np.ones((4, 1), dtype=int)  # [1;1;1;1]

S1 = 540  # 回路间距/mm
S2 = 302  # 回路内三角形腰长，或直线型相邻电缆间距/mm
S3 = 270  # 三角形底长/mm
GMRs = 47.6  # 电缆的金属护套的几何平均半径，/mm,可用护套的内外径的平均值代替

In [27]:
print('回路的类型:', typeCircuit)

回路的类型: [[1]
 [1]
 [1]
 [1]]


In [28]:
distance = cal_dis(row, column, typeCircuit, S1, S2, S3, GMRs)  # 各电缆间距

coorCable [[   0.            0.        ]
 [-135.         -270.14625668]
 [ 135.         -270.14625668]
 [ 540.            0.        ]
 [ 405.         -270.14625668]
 [ 675.         -270.14625668]
 [1080.            0.        ]
 [ 945.         -270.14625668]
 [1215.         -270.14625668]
 [1620.            0.        ]
 [1485.         -270.14625668]
 [1755.         -270.14625668]]


In [24]:
print('各电缆间距:', distance)

各电缆间距: [[  47.6         302.          302.          540.          486.83056601
   727.05158001 1080.          982.85502491 1244.67023745 1620.
  1509.37205486 1775.67001439]
 [ 302.           47.6         270.          727.05158001  540.
   810.         1244.67023745 1080.         1350.         1775.67001439
  1620.         1890.        ]
 [ 302.          270.           47.6         486.83056601  270.
   540.          982.85502491  810.         1080.         1509.37205486
  1350.         1620.        ]
 [ 540.          727.05158001  486.83056601   47.6         302.
   302.          540.          486.83056601  727.05158001 1080.
   982.85502491 1244.67023745]
 [ 486.83056601  540.          270.          302.           47.6
   270.          727.05158001  540.          810.         1244.67023745
  1080.         1350.        ]
 [ 727.05158001  810.          540.          302.          270.
    47.6         486.83056601  270.          540.          982.85502491
   810.         1080.        

In [15]:
length = np.zeros((1, 3))  # 交叉互联三段长度/km
length[0, 0] = 0.484  # 第一段长度
length[0, 1] = 0.484  # 第二段长度
length[0, 2] = 0.484  # 第三段长度
lengthAll = np.sum(length)  # 电缆总长/km

In [16]:
print('电缆总长:{0}km'.format(lengthAll))

电缆总长:1.452km


In [17]:
R1 = 0.001  # 一端口接地电阻/Ω
R2 = 0.001  # 二端口接地电阻/Ω
EarthResistivity = 400  # 大地电阻率/Ω*m
RSheathUint = 0.2076  # 护套单位电阻/Ω/km
IA = 412.2  # A相负载电流，默认相序为0

计算每一相电缆的负载电流

In [18]:
# 计算每一相电缆的负载电流
ILoad = np.zeros((numberCable, 1), dtype=complex)  # 电缆的负载电流/A，包含了实虚部,
for num in range(0, numberCable, 3):
    ILoad[num, 0] = IA  # A相负载电流
    ILoad[num + 1, 0] = -0.5 * IA - sqrt(3) / 2 * IA * 1j  # B相负载电流
    ILoad[num + 2, 0] = -0.5 * IA + sqrt(3) / 2 * IA * 1j  # C相负载电流

In [19]:
print('负载电流:', ILoad)

负载电流: [[ 412.2  +0.j        ]
 [-206.1-356.97567144j]
 [-206.1+356.97567144j]
 [ 412.2  +0.j        ]
 [-206.1-356.97567144j]
 [-206.1+356.97567144j]
 [ 412.2  +0.j        ]
 [-206.1-356.97567144j]
 [-206.1+356.97567144j]
 [ 412.2  +0.j        ]
 [-206.1-356.97567144j]
 [-206.1+356.97567144j]]


计算右端感应电压

In [21]:
print(distance)
print(GMRs)
frequency = 50  # 负载电流频率
w = 2 * pi * frequency
ULoad = np.zeros((numberCable, 1), dtype=complex)
fluxAll = np.zeros((numberCable, 1), dtype=complex)  # 计算每一根电缆的总磁通/(Wb/km)
E = np.zeros((numberCable, 1), dtype=complex)  # 每根电缆单位长度的感应电动势/V/km
for num in range(numberCable):
    for j in range(numberCable):
        fluxAll[num, 0] = fluxAll[num, 0] + 2 * (10**(-4)) * ILoad[j, 0] * log(
            distance[num, j] / GMRs)
    E[num, 0] = -w * fluxAll[num, 0] * 1j

for num in range(numberCircuit):  # 回路数
    for j in range(3):  # 相数
        if j == 0:
            ULoad[num * 3 + j,
                  0] = E[0 + 3 * num, 0] * length[0, 0] + E[1 + 3 * num][
                      0] * length[0, 1] + E[2 + 3 * num, 0] * length[0, 2]
        elif j == 1:
            ULoad[num * 3 + j, 0] = E[1 + 3 * num, 0] * length[0, 0] + E[
                2 + 3 * num] * length[0, 1] + E[0 + 3 * num, 0] * length[0, 2]
        elif j == 2:
            ULoad[num * 3 + j, 0] = E[2 + 3 * num, 0] * length[0, 0] + E[
                0 + 3 * num] * length[0, 1] + E[1 + 3 * num, 0] * length[0, 2]

UR = ULoad.real  # 负载感应电压实部
UF = ULoad.imag  # 负载感应电压虚部
U = np.transpose([np.append(UR, UF)])  # 6n×1

[[  47.6         302.          302.          540.          486.83056601
   727.05158001 1080.          982.85502491 1244.67023745 1620.
  1509.37205486 1775.67001439]
 [ 302.           47.6         270.          727.05158001  540.
   810.         1244.67023745 1080.         1350.         1775.67001439
  1620.         1890.        ]
 [ 302.          270.           47.6         486.83056601  270.
   540.          982.85502491  810.         1080.         1509.37205486
  1350.         1620.        ]
 [ 540.          727.05158001  486.83056601   47.6         302.
   302.          540.          486.83056601  727.05158001 1080.
   982.85502491 1244.67023745]
 [ 486.83056601  540.          270.          302.           47.6
   270.          727.05158001  540.          810.         1244.67023745
  1080.         1350.        ]
 [ 727.05158001  810.          540.          302.          270.
    47.6         486.83056601  270.          540.          982.85502491
   810.         1080.        ]
 [108

In [13]:
print('负载感应电压:', U)

负载感应电压: [[ 29.80625509]
 [ 29.80625509]
 [ 29.80625509]
 [  8.1092106 ]
 [  8.1092106 ]
 [  8.1092106 ]
 [ -8.1092106 ]
 [ -8.1092106 ]
 [ -8.1092106 ]
 [-29.80625509]
 [-29.80625509]
 [-29.80625509]
 [ -5.43309975]
 [ -5.43309975]
 [ -5.43309975]
 [ -8.14239017]
 [ -8.14239017]
 [ -8.14239017]
 [ -8.14239017]
 [ -8.14239017]
 [ -8.14239017]
 [ -5.43309975]
 [ -5.43309975]
 [ -5.43309975]]


计算R矩阵

In [14]:
# 计算R矩阵
De = sqrt(exp(12.981) * EarthResistivity / frequency) * 1000  # 护套等值深度/mm
REarth = (pi**2) * frequency * 0.0001 * (lengthAll + 2 * De * (10**(-6)))
RA = R1 + R2 + REarth + RSheathUint * lengthAll
RB = R1 + R2 + REarth
R = np.zeros((numberCable, numberCable))  # 3n×3n
for num in range(numberCable):
    if num % 3 == 0:  # A相
        R[num, num] = RA
        R[num, num + 1] = RB
        R[num, num + 2] = RB
    elif num % 3 == 1:  # B相
        R[num, num] = RA
        R[num, num + 1] = RB
        R[num, num - 1] = RB
    elif num % 3 == 2:  # C相
        R[num, num] = RA
        R[num, num - 1] = RB
        R[num, num - 2] = RB

In [15]:
print('R:', R)

R: [[0.55901027 0.25757507 0.25757507 0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.25757507 0.55901027 0.25757507 0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.25757507 0.25757507 0.55901027 0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.55901027 0.25757507 0.25757507
  0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.25757507 0.55901027 0.25757507
  0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.25757507 0.25757507 0.55901027
  0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.55901027 0.25757507 0.25757507 0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.257575

计算Xa，即X的元素,对角线为自感,注意都要乘以长度

In [16]:
# 计算Xa，即X的元素,对角线为自感,注意都要乘以长度
X = np.zeros((numberCable, numberCable))  # 感抗矩阵，3n×3n
for num1 in range(numberCable):  # 计算第num1根电缆的X矩阵元素
    for num2 in range(numberCable):  # 计算第num2根电缆Isnum2的Xa系数
        # 先判断num1和num2分别是哪一相电缆，0到2表示A到C
        temp1 = num1 % 3
        temp2 = num2 % 3
        # 根据判断结果分类计算X
        if temp1 == temp2:  # 如果都是同一相
            if num1 != num2:  # 如果不是同一根电缆
                X[num1, num2] = 2 * w * 10**(-4) * log(
                    De / distance[num1, num2]) * lengthAll
            else:  # 相等对角线，为自感
                X[num1, num2] = 2 * w * 10**(-4) * log(De / GMRs) * lengthAll
        else:  # 如果不是同一相电缆
            XTemp = np.zeros((1, 3))  # 与三段长度对应的单位互感
            if temp1 == 0 and temp2 == 1:  # A与B
                XTemp[0, 0] = 2 * w * 10**(-4) * log(
                    De / distance[num1, num2])  # AB
                XTemp[0, 1] = 2 * w * 10**(-4) * log(
                    De / distance[num1 + 1, num2 + 1])  # BC
                XTemp[0, 2] = 2 * w * 10**(-4) * log(
                    De / distance[num1 + 2, num2 - 1])  # CA
            if temp1 == 0 and temp2 == 2:  # A与C
                XTemp[0, 0] = 2 * w * 10**(-4) * log(
                    De / distance[num1, num2])  # AC
                XTemp[0, 1] = 2 * w * 10**(-4) * log(
                    De / distance[num1 + 1, num2 - 2])  # BA
                XTemp[0, 2] = 2 * w * 10**(-4) * log(
                    De / distance[num1 + 2, num2 - 1])  # CB
            if temp1 == 1 and temp2 == 0:  # B与A
                XTemp[0, 0] = 2 * w * 10**(-4) * log(
                    De / distance[num1, num2])  # BA
                XTemp[0, 1] = 2 * w * 10**(-4) * log(
                    De / distance[num1 + 1, num2 + 1])  # CB
                XTemp[0, 2] = 2 * w * 10**(-4) * log(
                    De / distance[num1 - 1, num2 + 2])  # AC
            if temp1 == 1 and temp2 == 2:  # B与C
                XTemp[0, 0] = 2 * w * 10**(-4) * log(
                    De / distance[num1, num2])  # BC
                XTemp[0, 1] = 2 * w * 10**(-4) * log(
                    De / distance[num1 + 1, num2 - 2])  # CA
                XTemp[0, 2] = 2 * w * 10**(-4) * log(
                    De / distance[num1 - 1, num2 - 1])  # AB
            if temp1 == 2 and temp2 == 0:  # C与A
                XTemp[0, 0] = 2 * w * 10**(-4) * log(
                    De / distance[num1, num2])  # CA
                XTemp[0, 1] = 2 * w * 10**(-4) * log(
                    De / distance[num1 - 2, num2 + 1])  # AB
                XTemp[0, 2] = 2 * w * 10**(-4) * log(
                    De / distance[num1 - 1, num2 + 2])  # BC
            if temp1 == 2 and temp2 == 1:  # C与B
                XTemp[0, 0] = 2 * w * 10**(-4) * log(
                    De / distance[num1, num2])  # CB
                XTemp[0, 1] = 2 * w * 10**(-4) * log(
                    De / distance[num1 - 2, num2 + 1])  # AC
                XTemp[0, 2] = 2 * w * 10**(-4) * log(
                    De / distance[num1 - 1, num2 - 1])  # BA
            X[num1, num2] = XTemp[0, 0] * length[0, 0] + XTemp[0, 1] * length[
                0, 1] + XTemp[0, 2] * length[0, 2]

In [17]:
print('X', X)

X [[0.96478989 0.79963659 0.79963659 0.74321178 0.73718564 0.74620086
  0.67997468 0.67892143 0.68009215 0.64298335 0.64259756 0.64294741]
 [0.79963659 0.96478989 0.79963659 0.74620086 0.74321178 0.73718564
  0.68009215 0.67997468 0.67892143 0.64294741 0.64298335 0.64259756]
 [0.79963659 0.79963659 0.96478989 0.73718564 0.74620086 0.74321178
  0.67892143 0.68009215 0.67997468 0.64259756 0.64294741 0.64298335]
 [0.74321178 0.74620086 0.73718564 0.96478989 0.79963659 0.79963659
  0.74321178 0.73718564 0.74620086 0.67997468 0.67892143 0.68009215]
 [0.73718564 0.74321178 0.74620086 0.79963659 0.96478989 0.79963659
  0.74620086 0.74321178 0.73718564 0.68009215 0.67997468 0.67892143]
 [0.74620086 0.73718564 0.74321178 0.79963659 0.79963659 0.96478989
  0.73718564 0.74620086 0.74321178 0.67892143 0.68009215 0.67997468]
 [0.67997468 0.68009215 0.67892143 0.74321178 0.74620086 0.73718564
  0.96478989 0.79963659 0.79963659 0.74321178 0.73718564 0.74620086]
 [0.67892143 0.67997468 0.68009215 0.73

AI=U

In [19]:
# AI=U
A = np.vstack((np.hstack((R, -X)), np.hstack((X, R))))  # 6n×6n
I = np.matrix(A).I * U  # 6n×1，先所有环流的实部，再是说有的虚部
ICircular = np.zeros((numberCable, 1))  # 记录每根电缆的环流值,3n×1
for num in range(numberCable):
    ICircular[num, 0] = sqrt((I[num, 0]**2) + (I[num + numberCable, 0]**2))
    temp1 = floor(num / 3)  # 第几个回路
    temp2 = num % 3  # 该回路中ABC三相
    print('回路{0}中{1}相的环流为{2}A'.format(temp1, temp2, ICircular[num, 0]))

回路0中0相的环流为22.369115786452955A
回路0中1相的环流为22.369115786452905A
回路0中2相的环流为22.36911578645294A
回路1中0相的环流为6.418633588066145A
回路1中1相的环流为6.4186335880660765A
回路1中2相的环流为6.418633588066016A
回路2中0相的环流为6.182138574819549A
回路2中1相的环流为6.182138574819656A
回路2中2相的环流为6.182138574819632A
回路3中0相的环流为24.021865612065092A
回路3中1相的环流为24.021865612065444A
回路3中2相的环流为24.02186561206548A
